### Preset editor: Operator (FM)

This notebook saves mapped parameter values back into Ableton Live’s native preset format. Because most closed‑source plugins do not document their preset serialization, we first decompress and inspect the preset structure and then write our values into the appropriate XML nodes.

In the context of the paper, `preset_editor_xx` denotes notebooks that perform this export step for different synthesizers/plugins. Here, `preset_editor_operator_fm` targets Ableton Operator, writing values computed in `parameter_operator_fm` to an Operator preset. We start from a blank/default preset (`dummy.adv`) and modify it to reproduce a target sound in Fixed Oscillator Mode or using a MIDI note as input. A comparison table verifies the written parameters.


### Workflow

- **Decompress preset**: Load the compressed `.adv` preset and decompress it to XML (`.xml`).
- **Load mapped values**: Read the tabular parameters from `tsv/final_values_fm_operator.tsv` generated by `parameter_operator_fm`.
- **Update XML nodes**: Locate each `Operator.i` block and set `Coarse/Manual@Value`, `Fine/Manual@Value`, and `Volume/Manual@Value` from the table.
- **Save updated preset XML**: Write to `xml_presets/updated_fmtest.xml` with XML declaration.
- **Verify changes**: Extract before/after values and display a comparison DataFrame inside the notebook output.
- **Re-compress to preset**: Gzip the updated XML back to `.adv` (e.g., `xml_presets/compressed_fmttest4.adv`) ready for loading in Ableton Live.

Note: The same pattern applies to other `preset_editor_xx` notebooks, with synthesizer‑specific node paths and parameter mappings.


In [4]:
import gzip

# Function to decompress a gzip file
def decompress_gzip(file_path, output_path):
    with gzip.open(file_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            f_out.write(f_in.read())

# Paths to the input and output files
input_path = 'xml_presets/dummy.adv'
output_path = 'xml_presets/dummy.xml'

# Decompress the gzip file
decompress_gzip(input_path, output_path)

print(f"Decompressed file saved to {output_path}")


Decompressed file saved to xml_presets/dummy.xml


In [5]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define the filename
tsv_filename = 'tsv/final_values_fm_operator.tsv'

# Read the DataFrame from the TSV file
df = pd.read_csv(tsv_filename, sep='\t')

# Extract new values from DataFrame
new_values = {
    f"Operator.{i}": {
        "coarse": int(df.loc[i, "Coarse Tuning"]),
        "fine": int(df.loc[i, "Fine Tuning"]),
        "volume": df.loc[i, "Level"]
    } for i in range(len(df))
}

# Load the XML file
file_path = 'xml_presets/dummy.xml'
tree = ET.parse(file_path)
root = tree.getroot()

# Update the Algorithm to 4 chained oscillators (Value 0)
algorithm = root.find('.//Globals/Algorithm/Manual')
if algorithm is not None:
    algorithm.set('Value', '0')
    print(f"Algorithm set to 0 (4 chained oscillators)")
else:
    print("Warning: Algorithm tag not found in XML.")

# Update the values for each operator
for operator_tag, values in new_values.items():
    operator = root.find(f'.//{operator_tag}')
    if operator is not None:
        # Update coarse
        coarse = operator.find('.//Coarse/Manual')
        if coarse is not None:
            coarse.set('Value', str(values["coarse"]))
        
        # Update fine
        fine = operator.find('.//Fine/Manual')
        if fine is not None:
            fine.set('Value', str(values["fine"]))
        
        # Update volume
        volume = operator.find('.//Volume/Manual')
        if volume is not None:
            volume.set('Value', str(values["volume"]))
    else:
        print(f"{operator_tag} not found")

# Save the updated XML to a new file with XML declaration
updated_file_path = 'xml_presets/updated_fmtest.xml'
tree.write(updated_file_path, xml_declaration=True, encoding='utf-8', method="xml")

# Print the path of the saved XML file
print(f"Updated XML file saved at: {updated_file_path}")

# Function to extract values from the XML for comparison
def extract_values(root):
    values = {}
    for i in range(4):
        operator_tag = f'Operator.{i}'
        operator = root.find(f'.//{operator_tag}')
        if operator is not None:
            coarse = operator.find('.//Coarse/Manual')
            fine = operator.find('.//Fine/Manual')
            volume = operator.find('.//Volume/Manual')
            if coarse is not None and fine is not None and volume is not None:
                values[operator_tag] = {
                    'coarse': coarse.get('Value'),
                    'fine': fine.get('Value'),
                    'volume': volume.get('Value')
                }
            else:
                print(f"Missing elements in {operator_tag}")
        else:
            print(f"Operator tag {operator_tag} not found")
    return values

# Load the original XML file again to get the original values
original_tree = ET.parse(file_path)
original_root = original_tree.getroot()

# Load the updated XML file
updated_tree = ET.parse(updated_file_path)
updated_root = updated_tree.getroot()

# Extract values from both original and updated XML
original_values = extract_values(original_root)
updated_values = extract_values(updated_root)

# Create a DataFrame to compare original and updated values
comparison_df = pd.DataFrame({
    "Original Coarse": {op: original_values[op]['coarse'] for op in original_values},
    "Updated Coarse": {op: updated_values[op]['coarse'] for op in updated_values},
    "Original Fine": {op: original_values[op]['fine'] for op in original_values},
    "Updated Fine": {op: updated_values[op]['fine'] for op in updated_values},
    "Original Volume": {op: original_values[op]['volume'] for op in original_values},
    "Updated Volume": {op: updated_values[op]['volume'] for op in updated_values}
})

# Display the comparison DataFrame
comparison_df


Algorithm set to 0 (4 chained oscillators)
Updated XML file saved at: xml_presets/updated_fmtest.xml


,Original Coarse,Updated Coarse,Original Fine,Updated Fine,Original Volume,Updated Volume
Operator.0,1,1,0,0,0.1258925349,0.7224452697013005
Operator.1,1,0,24,17,0.1000000015,0.3212918029357252
Operator.2,1,2,514,770,0.1083927006,0.0220896287220672
Operator.3,0,5,13,115,0.5623413324,0.0951017043351228


In [6]:
import gzip

# Function to compress an XML file to gzip format
def compress_to_gzip(input_path, output_path):
    with open(input_path, 'rb') as f_in:
        with gzip.open(output_path, 'wb') as f_out:
            f_out.writelines(f_in)

# Paths to the input (edited XML) and output (compressed) files
input_path = 'xml_presets/updated_fmtest.xml'  # Your edited XML file
output_path = 'xml_presets/compressed_fmttest5.adv'   # The gzip output file

# Compress the XML file to gzip format
compress_to_gzip(input_path, output_path)

print(f"Compressed file saved to {output_path}")


Compressed file saved to xml_presets/compressed_fmttest5.adv
